In [1]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim

In [2]:
# Step 1: Generate Dataset
def generate_dataset(num_samples):
    np.random.seed(42)
    coordinates = np.random.randint(0, 100, size=(num_samples, 4))  # Generate random coordinates (x1, y1, x2, y2)
    distances = np.sum(np.abs(coordinates[:, :2] - coordinates[:, 2:]), axis=1)
    return coordinates, distances

In [3]:
# Step 2: Preprocess Data
def preprocess_data(coordinates, distances):
    return coordinates.astype(float), distances.astype(float)

In [4]:
# Step 3: Design Neural Network
class DistancePredictor(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(DistancePredictor, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        out = self.fc1(x)
        out = self.relu(out)
        out = self.fc2(out)
        return out

In [5]:
# Step 4: Train Neural Network
def train_model(model, X_train, y_train, num_epochs=100, learning_rate=0.01):
    criterion = nn.MSELoss()
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)

    for epoch in range(num_epochs):
        inputs = torch.tensor(X_train, dtype=torch.float)
        labels = torch.tensor(y_train, dtype=torch.float)

        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        if (epoch+1) % 10 == 0:
            print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

In [6]:
# Step 5: Find Pair with Smallest Predicted Distance
def find_smallest_distance(model, coordinates):
    inputs = torch.tensor(coordinates, dtype=torch.float)
    distances = model(inputs).detach().numpy().squeeze()
    min_distance_index = np.argmin(distances)
    return coordinates[min_distance_index]

In [7]:
# Custom num_samples
num_samples = 1000

# Generate dataset
coordinates, distances = generate_dataset(num_samples)

In [8]:
# Preprocess data
X_train, y_train = preprocess_data(coordinates, distances)

In [9]:
# Design neural network
input_size = X_train.shape[1]
hidden_size = 64
output_size = 1
model = DistancePredictor(input_size, hidden_size, output_size)

In [10]:
# Train model
train_model(model, X_train, y_train)

/usr/local/lib/python3.10/dist-packages/torch/nn/modules/loss.py:535: UserWarning: Using a target size (torch.Size([1000])) that is different to the input size (torch.Size([1000, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


Epoch [10/100], Loss: 1950.1400
Epoch [20/100], Loss: 1535.2948
Epoch [30/100], Loss: 1421.0089
Epoch [40/100], Loss: 1383.3336
Epoch [50/100], Loss: 1364.6008
Epoch [60/100], Loss: 1357.7064
Epoch [70/100], Loss: 1351.9795
Epoch [80/100], Loss: 1349.1238
Epoch [90/100], Loss: 1346.8910
Epoch [100/100], Loss: 1345.1086


In [11]:
# Find pair with smallest predicted distance
pair_with_smallest_distance = find_smallest_distance(model, coordinates)
print("Pair with smallest predicted distance:", pair_with_smallest_distance)

Pair with smallest predicted distance: [2 5 4 4]
